# Movie Recommendation System 

## Loading the data

 MovieLens datasets : size 1 million 

In [1]:

import numpy as np
import pandas as pd
import os
import time
from collections import Counter



movies_data = pd.read_excel('data/movies.xlsx')
ratings_data = pd.read_excel('data/ratings.xlsx')
user_data = pd.read_excel('data/users.xlsx')





In [37]:
movies_data_head = movies_data.head()
ratings_data_head = ratings_data.head()
print(movies_data_head.columns)
print(ratings_data_head.columns)
print(user_data.columns)



Index(['movie_id', 'title', 'genre'], dtype='object')
Index(['user_id', 'movie_id', 'rating', 'time_stamp'], dtype='object')
Index(['user_id', 'gender', 'age', 'job', 'zipcode'], dtype='object')


## Data preprocessing

This is being done to improve the efficiency  of the system, rather than making few calculations at run time, we are making few calculations before had like average rating of the film, what movies are watched by which user, how is he rating films, he last seen move genre and keeping a record how people of different age, gender and job watch which films . 

In [38]:



movie_id_data = {}  # movie_id :: {ratings::[list] -> avg, genere }
genre_based_data = {}  # genere :: movie_id::
for j, row in movies_data.iterrows():
    movie_id_data[row[0]] = {'ratings': [], 'genre': row[2].split('|'), 'title': row[1]}
    for i in row[2].split('|'):
        if i not in genre_based_data.keys():
            genre_based_data[i] = []
            genre_based_data[i].append(row[0])
        else:
            genre_based_data[i].append(row[0])




# user based data
user_id_info = {}  # user_id::{ age,job,sex }
for i, row in user_data.iterrows():
    user_id_info[row[0]] = {'gender': row[1], 'age': row[2], 'job': row[3]}



In [39]:
user_id_ratings_data = {}  # user_id :: Ratings:[movie_Id] list
user_id_movie_data = {}  # user_id :: movie_id :: ratings
user_id_viewing_data = {}  # user_id ::genere:count
user_id_viewing_history = {}  # user_id :: [time_stamp: max , movie_id ]
age_gender_movies = {}  # age:: sex :: movie_id::count
job_data = {} #job:: movie Id:: count
movie_based_dict = {}  # movie_id: user_id: ratings
movie_id_view_count = {}
cosine_similarity_hash ={} # user_id:: user_id:: similarity
for i, row in ratings_data.iterrows(): #removed head
    ##############################################
    if row[0] not in user_id_ratings_data.keys():
        user_id_ratings_data[row[0]] = {}  # this will be used to find the moves user has rated the highest
        user_id_ratings_data[row[0]] = {1: [], 2: [], 3: [], 4: [], 5: []}
        user_id_ratings_data[row[0]][row[2]].append(row[1])
    else:
        user_id_ratings_data[row[0]][row[2]].append(row[1])
    #####################################################
    if row.iloc[0] not in user_id_movie_data.keys():
        user_id_movie_data[row.iloc[0]] = {}  # based on user Id : movie_id for calculating cosine distance
        user_id_movie_data[row.iloc[0]][row.iloc[1]] = row.iloc[2]
    else:
        user_id_movie_data[row.iloc[0]][row.iloc[1]] = row.iloc[2]
    #####################################################
    movie_id_data[row[1]]['ratings'].append(row[2])  # it is done to calculate the mean rating for a film
    #################################################
    if row.iloc[1] not in movie_based_dict.keys():
        movie_based_dict[row.iloc[1]] = {}
        movie_based_dict[row.iloc[1]][row.iloc[0]] = row.iloc[2]
    else:
        movie_based_dict[row.iloc[1]][row.iloc[0]] = row.iloc[2]
    ################################################
    if row[0] not in user_id_viewing_history.keys():  # this is to find the last seen film by user,
        user_id_viewing_history[row[0]] = {'time_stamp': row[3], 'movie_id': row[1]}
    else:
        if user_id_viewing_history[row[0]]['time_stamp'] < row[3]:
            user_id_viewing_history[row[0]]['movie_id'] = row[1]
    ######################################################
    if row[0] not in user_id_viewing_data.keys():  # used to find the film genre most watched by user
        user_id_viewing_data[row[0]] = {}
    genre_of_film = movie_id_data[row[1]]['genre']
    for j in genre_of_film:
        if j not in user_id_viewing_data[row[0]].keys():
            user_id_viewing_data[row[0]][j] = 1
        else:
            user_id_viewing_data[row[0]][j] += 1
            #######################################################
    if user_id_info[row[0]][
        'age'] not in age_gender_movies.keys():  # this is used to recommend movie based on age::gender
        age_gender_movies[user_id_info[row[0]]['age']] = {'M': {}, 'F': {}}
    if row[1] not in age_gender_movies[user_id_info[row[0]]['age']][user_id_info[row[0]]['gender']].keys():
        age_gender_movies[user_id_info[row[0]]['age']][user_id_info[row[0]]['gender']][row[1]] = 1
    else:
        age_gender_movies[user_id_info[row[0]]['age']][user_id_info[row[0]]['gender']][row[1]] += 1
        #######################################################
    if user_id_info[row[0]]['job'] not in job_data.keys():
        job_data[user_id_info[row[0]]['job']] = {}
    if row[1] not in job_data[user_id_info[row[0]]['job']].keys():
        job_data[user_id_info[row[0]]['job']][row[1]] = {}
        job_data[user_id_info[row[0]]['job']][row[1]] = 1
    else:
        job_data[user_id_info[row[0]]['job']][row[1]] += 1


In [40]:


# find average rating for film
for i in movie_id_data.keys():
    if len(movie_id_data[i]['ratings']) > 0:
        movie_id_view_count[i] = len(movie_id_data[i]['ratings'])
        movie_id_data[i]['mean'] = np.mean(movie_id_data[i]['ratings'])
    else:
        movie_id_data[i]['mean'] = []
        


## Defining the functions needed to get recommendations  

### I intend to give recommendations of the films based on the following factors,<br>
•	Cosine similarity [Collaboration filtering ] <br>
•	Genre of the film most watched by user<br> 
•	Genre of the film last seen by user <br> 
•	Age and Gender of user<br> 
•	Job of the user<br> 

While recommending if we have two films having similar stats, we recommend the film with highest average rating and views.  if the system say’s “No recommendations in this category”, It means that the user has seen all the films in that category. I have taken precaution that if a movie is recommended in one category, it should not be recommended another category. 

#### Recommendations based on Cosine similarity [Collaboration filtering ] 
The functions below are used to calculate cosine similarity, although we have this function in Sklearn, while doing this project I learned it is quite slow and when  define a function by our self we improve the performance by many magnitudes .    

The functions below are used to calculate cosine similarity, although we have this function in Sklearn, while doing this project I learned it is quite slow and when define a function by our self we improve the performance by many magnitudes.    
When calculating the cosine similarity, we use vectors of ratings given by users, they are organized in the following format. The ratings are of corresponding move ids in ascending order, if rating 0 means the film is not watched 

User 1:     4 5 0 4 0 0 1 <br>
User 2:     1 0  1 0 0 0 1<br>
I have modified them to this<br> 
User 1:	   4 5 0 4 1<br>
User 2:    1 0 1 0 1<br><br>

By doing this I increased the performance of this system, if we have not we will have to deal with vectors of length around 6000 and in real life cases it can be of many times larger. <br>

By doing this I increased the performance of this system, if we have not we will have to deal with vectors of length around 6000 and in real life cases it can be of many times larger.  When the length of the vector keeps increasing its it takes more time to recommend moves using cosine similarity. Use of GPU’s and building the system using a compiled language should improve the performance of the systems by many fold’s.  



In [41]:
import operator,math
def dot_product2(v1, v2):
    return sum(map(operator.mul, v1, v2))

def cosine_similarity(i, j):
    a1 = Counter(user_id_movie_data[i])
    a2 = dict.fromkeys(user_id_movie_data[i], np.mean(list(user_id_movie_data[i].values())))
    t1 = {key: a1[key] - a2.get(key, 0) for key in a1.keys()}
    a1 = Counter(user_id_movie_data[j])
    a2 = dict.fromkeys(user_id_movie_data[j], np.mean(list(user_id_movie_data[j].values())))
    t2 = {key: a1[key] - a2.get(key, 0) for key in a1.keys()}
    d1 = dict.fromkeys(
            set(user_id_movie_data[i].keys()).intersection(user_id_movie_data[j].keys()).symmetric_difference(
                user_id_movie_data[j].keys()), 0)
    d2 = dict.fromkeys(
            set(user_id_movie_data[j].keys()).intersection(user_id_movie_data[i].keys()).symmetric_difference(
                user_id_movie_data[i].keys()), 0)
    t1.update(d1)
    t2.update(d2)
    key_temp = t1.keys()
    t1 = list(map(lambda x: t1[x], key_temp))
    t2 = list(map(lambda x: t2[x], key_temp))     
    prod = dot_product2(t1, t2)
    len1 = math.sqrt(dot_product2(t1, t1))
    len2 = math.sqrt(dot_product2(t2, t2))
    return prod / (len1 * len2)

In [42]:
current_recommended_movie_id = set()

In [43]:
past_recommendations = set()

In [44]:
def recommend_similarity(current_id, current_recommended_movie_id, past_recommendations):
    movies_rated_high = []
    rating = 5
    while len(movies_rated_high) < 1:
        movies_rated_high += user_id_ratings_data[current_id][rating]
        rating -= 1
        movies_rated_high = set(movies_rated_high).difference(current_recommended_movie_id)
        movies_rated_high = set(movies_rated_high).difference(past_recommendations)
        movies_rated_high = list(movies_rated_high)
        if rating == 0:
            break
    if len(movies_rated_high) == 0:
        movies_rated_high = -1
    potential_similar_user_ids = set()
    for i in movies_rated_high:
        potential_similar_user_ids.update(movie_based_dict[i].keys())
    i = current_id
    potential_similar_user_ids.discard(i)
    similarity_dict = {}        
    for j in potential_similar_user_ids:     
        # have a similarity block
        if i in cosine_similarity_hash.keys():
            if j  in cosine_similarity_hash[i].keys():
                similarity_dict[j] = cosine_similarity_hash[i][j]
            else:
                similarity_dict[j] = cosine_similarity(i, j)
                cosine_similarity_hash[i][j] ={}
                if j not in cosine_similarity_hash.keys():
                    cosine_similarity_hash[j] ={}                                        
                cosine_similarity_hash[j][i] ={}
                cosine_similarity_hash[i][j] = cosine_similarity_hash[j][i] = similarity_dict[j]
        else:
            similarity_dict[j] = cosine_similarity(i, j)
            cosine_similarity_hash[i] ={}
            cosine_similarity_hash[i][j] ={}
            cosine_similarity_hash[j] ={}
            cosine_similarity_hash[j][i] ={}
            cosine_similarity_hash[i][j] = cosine_similarity_hash[j][i] = similarity_dict[j]
    max_val = max(similarity_dict.values())
    user_max_similarity = [key for key, val in similarity_dict.items() if val == max_val]
    movies_watched_by_similar_user = set()
    for i in user_max_similarity:
        temp_rating = rating + 1
        while len(movies_watched_by_similar_user) < 1:
            movies_watched_by_similar_user.update(
                user_id_ratings_data[i][temp_rating])  # movies watched and rated similar
            movies_watched_by_similar_user = movies_watched_by_similar_user.difference(current_recommended_movie_id)
            movies_watched_by_similar_user = movies_watched_by_similar_user.difference(past_recommendations)
            movies_watched_by_similar_user = movies_watched_by_similar_user.difference(user_id_movie_data[current_id].keys())
            temp_rating -= 1
            if temp_rating == 0:
                break
    potential_recommendation_by_similarity = movies_watched_by_similar_user
    
    temp = {}
    #print(potential_recommendation_by_similarity)
    for i in potential_recommendation_by_similarity:
        temp[i] = movie_id_data[i]['mean']  # picking one with highest ratings
    try:    
        max_val = max(temp.values())
        potential_recommendations = [key for key, val in temp.items() if val == max_val]
        temp = {}
        flag = 0
        # you need to add this block to everything else
        for j in potential_recommendations:
            temp[j] = movie_id_view_count[j]  
        if len(temp.values()) != 0:
            max_val = max(temp.values())
            recommendations_by_similarity = list(set([key for key, val in temp.items() if val == max_val]))[0]
        else:
            flag = 1
        if flag == 0:
            current_recommended_movie_id.update([recommendations_by_similarity])
            t1 = movie_id_data[recommendations_by_similarity]['title']
        else:
            recommendations_by_similarity = 'No recommendations in this category'
            t1 = recommendations_by_similarity
        return recommendations_by_similarity
    except ValueError:
        return 'No recommendations in this category'

    

### Recommendations based on Genre most watched

From here on recommending for the remaining categories is very easy and straight forward. In the preprocessing stage we calculated the genre which is most watched by the user. So, we recommend the film in that genre that is not watched by user and the one with highest average rating and most watched film. 

In [45]:
def recommend_genre(current_id, current_recommended_movie_id, past_recommendations):
    max_val = max(user_id_viewing_data[current_id].values())
    genre_most_watched = [key for key, val in user_id_viewing_data[current_id].items() if val == max_val]
    movie_list = []
    for i in genre_most_watched:
        movie_list += genre_based_data[i]
    movie_list = set(movie_list)
    movie_list = movie_list.difference(
        user_id_movie_data[current_id].keys())  # removes the film's that were already seen
    ###############################################
    # here you can add a block that can remove all the films that are previously recomended and the films that were previosly seen
    ##############################################
    temp = {}
    for i in range(0, len(movie_list)):
        try:
            if len(movie_id_data[list(movie_list)[i]]['mean']) == 0:
                temp[list(movie_list)[i]] = 0
        except TypeError:
            temp[list(movie_list)[i]] = movie_id_data[list(movie_list)[i]]['mean']

    max_val = max(temp.values())  # taking films with high ratings
    potential_recommendations_by_genre = set([key for key, val in temp.items() if val == max_val])
    potential_recommendations = potential_recommendations_by_genre.difference(current_recommended_movie_id)
    potential_recommendations = potential_recommendations.difference(past_recommendations)
    potential_recommendations = potential_recommendations.difference(user_id_movie_data[current_id].keys())
    temp = {}
    flag = 0
    # you need to add this block to everything else
    for j in potential_recommendations:
        temp[j] = movie_id_view_count[j]
    if len(temp.values()) != 0:
        max_val = max(temp.values())
        recommendations_by_genre = list(set([key for key, val in temp.items() if val == max_val]))[0]
    else:
        flag = 1
    if flag == 0:
        current_recommended_movie_id.update([recommendations_by_genre])

    else:
        recommendations_by_genre = 'No recommendations in this category'
    return recommendations_by_genre


### Recommendations based on Genre of the film last watched

In the preprocessing stage we calculated the genre which is last watched by the user. So, we recommend the film in that genre that is not watched by user and the one with highest average rating and most watched film.

In [46]:
def last_watched(current_id, current_recommended_movie_id, past_recommendations):
    genre_last_watched = movie_id_data[user_id_viewing_history[current_id]['movie_id']]['genre']
    movie_list = []
    for i in genre_last_watched:
        movie_list += genre_based_data[i]
    movie_list = set(movie_list)
    movie_list = movie_list.difference(
        user_id_movie_data[current_id].keys())  # removes the film's that were already seen
    movie_list = movie_list.difference(current_recommended_movie_id)
    movie_list = movie_list.difference(past_recommendations)
    # above line removes the films that are currently being recommended
    ###############################################
    # here you can add a block that can remove all the films that are previously recomended and the films that were previosly seen
    ##############################################
    temp = {}
    for i in range(0, len(movie_list)):
        try:
            if len(movie_id_data[list(movie_list)[i]]['mean']) == 0:
                temp[list(movie_list)[i]] = 0
        except TypeError:
            temp[list(movie_list)[i]] = movie_id_data[list(movie_list)[i]]['mean']
    max_val = max(temp.values())
    potential_recommendations = set([key for key, val in temp.items() if val == max_val])
    temp = {}
    flag = 0
    # you need to add this block to everything else
    for j in potential_recommendations:
        temp[j] = movie_id_view_count[j]
    if len(temp.values()) != 0:
        max_val = max(temp.values())
        recommendations_by_last_viewed_film = list(set([key for key, val in temp.items() if val == max_val]))[0]
    else:
        flag = 1
    if flag == 0:
        current_recommended_movie_id.update([recommendations_by_last_viewed_film])
    else:
        recommendations_by_last_viewed_film = 'No recommendations in this category'
    return recommendations_by_last_viewed_film


### Recommendations based on age and gender of the user

In the preprocessing stage we calculated which age group users belonging to which gender watch which film .So, we recommend the film that is not watched by user and the one with highest average rating and most watched film.

In [47]:
def recommend_age_gender(current_id,current_recommended_movie_id,past_recommendations ):
    age=user_id_info[current_id]['age']
    gender = user_id_info[current_id]['gender']
    potential_recommendations = age_gender_movies[age][gender].copy()
    movie_list = set()
    movie_list = movie_list.union(
        user_id_movie_data[current_id].keys())  # removes the film's that were already seen
    movie_list = movie_list.union(current_recommended_movie_id)
    movie_list = movie_list.union(past_recommendations)
    # here you can take a union of current_recommended moives and previouly recommended movies and what one has already seen 
    ###########################
    for j in movie_list:
        try:
            del potential_recommendations[j]
        except KeyError:
            pass
    temp = {}
    flag = 0
    # you need to add this block to everything else
    for j in potential_recommendations:
        temp[j] =movie_id_view_count[j]
    if len(temp.values()) != 0:    
        max_val = max(temp.values())    
        recommendations_by_age_gender= list(set([key for key, val in temp.items() if val == max_val]))[0]
    else:
        flag = 1    
    if flag  == 0:
        current_recommended_movie_id.update([recommendations_by_age_gender])
        t4 = movie_id_data[recommendations_by_age_gender]['title']
    else:
        recommendations_by_age_gender = 'No recommendations in this category'
        t4 = recommendations_by_age_gender
    return recommendations_by_age_gender    

### Recommendations based on job of the user


In the preprocessing stage we calculated, users having which job watch which films. So, we recommend the film that is  not watched by user and the one with highest average rating and most watched film.


In [48]:
def recommend_job(current_id,current_recommended_movie_id,past_recommendations ):
    job_id = user_id_info[current_id]['job']
    movies_watched_by_profession = set(job_data[job_id].keys())
    potential_recommendations =  movies_watched_by_profession.difference(current_recommended_movie_id)
    potential_recommendations =  potential_recommendations.difference(past_recommendations)
    # need to remove those films that are already recommended 
    potential_recommendations = potential_recommendations.difference(user_id_movie_data[current_id].keys())
    temp = {}
    flag = 0
    # you need to add this block to everything else
    for j in potential_recommendations:
        temp[j] =movie_id_view_count[j]
    if len(temp.values()) != 0:    
        max_val = max(temp.values())    
        recommendations_by_profession= list(set([key for key, val in temp.items() if val == max_val]))[0]  
    else:
        flag = 1
    if flag  == 0:
        current_recommended_movie_id.update([recommendations_by_profession])
        t5 = movie_id_data[recommendations_by_profession]['title']
    else:
        recommendations_by_profession = 'No recommendations in this category'
        t5 = recommendations_by_profession   
    return recommendations_by_profession    

### Geting output

bellow I have written recommendations for every user and store them to an excel sheet, which can been seen as part of the repo this notebook is a part of. do not run it with whole data set loaded as it will take lot of time to get recommendations. 

In [49]:
def get_title(r1,r2,r3,r4,r5):
    if r1 != 'No recommendations in this category':
        t1 = movie_id_data[r1]['title']
    else:
        t1 = r1
    if r2 != 'No recommendations in this category':
        t2 = movie_id_data[r2]['title']   
    else:
        t2 = r2   
    if r3 != 'No recommendations in this category':
        t3 = movie_id_data[r3]['title']
    else:
        t3 = r3    
    if r4 != 'No recommendations in this category':
        t4 = movie_id_data[r4]['title']
    else:
        t4 = r4    
    if r5 != 'No recommendations in this category':
        t5 = movie_id_data[r5]['title']
    else:
        t5 = r5
    return t1,t2,t3,t4,t5


In [50]:
if __name__ == "__main__":
    c1 = 'user_id'
    c2='recommendations_by_similarity'
    c3='recommendations_by_genre'
    c4='recommendations_by_last_viewed_film'
    c5='recommendations_by_age_gender'
    c6='recommendations_by_profession'
    df = pd.DataFrame(columns=[c1,c2,c3,c4,c5])
    for i in user_id_ratings_data.keys():
        current_recommended_movie_id =set()
        recommendations_by_similarity = recommend_similarity(i, current_recommended_movie_id, past_recommendations)
        recommendations_by_genre = recommend_genre(i, current_recommended_movie_id, past_recommendations)
        recommendations_by_last_viewed_film = last_watched(i, current_recommended_movie_id, past_recommendations)
        recommendations_by_age_gender = recommend_age_gender(i, current_recommended_movie_id, past_recommendations)
        recommendations_by_profession = recommend_job(i, current_recommended_movie_id, past_recommendations)
        
        r1 = recommendations_by_similarity
        r2 = recommendations_by_genre
        r3 = recommendations_by_last_viewed_film
        r4 = recommendations_by_age_gender
        r5 = recommendations_by_profession
        t1, t2, t3, t4, t5 = get_title(r1, r2, r3, r4, r5)
        df = df.append({c1: int(i),c2: t1,c3: t2,c4: t3,c5: t4,c6: t5}, ignore_index=True)
        print('getting recommendations for user: ',i)

getting recommendations for user:  1
getting recommendations for user:  2
getting recommendations for user:  3
getting recommendations for user:  4
getting recommendations for user:  5
getting recommendations for user:  6
getting recommendations for user:  7
getting recommendations for user:  8
getting recommendations for user:  9
getting recommendations for user:  10
getting recommendations for user:  11
getting recommendations for user:  12
getting recommendations for user:  13
getting recommendations for user:  14
getting recommendations for user:  15
getting recommendations for user:  16
getting recommendations for user:  17
getting recommendations for user:  18
getting recommendations for user:  19
getting recommendations for user:  20
getting recommendations for user:  21
getting recommendations for user:  22
getting recommendations for user:  23
getting recommendations for user:  24
getting recommendations for user:  25
getting recommendations for user:  26
getting recommendatio

getting recommendations for user:  214
getting recommendations for user:  215
getting recommendations for user:  216
getting recommendations for user:  217
getting recommendations for user:  218
getting recommendations for user:  219
getting recommendations for user:  220
getting recommendations for user:  221
getting recommendations for user:  222
getting recommendations for user:  223
getting recommendations for user:  224
getting recommendations for user:  225
getting recommendations for user:  226
getting recommendations for user:  227
getting recommendations for user:  228
getting recommendations for user:  229
getting recommendations for user:  230
getting recommendations for user:  231
getting recommendations for user:  232
getting recommendations for user:  233
getting recommendations for user:  234
getting recommendations for user:  235
getting recommendations for user:  236
getting recommendations for user:  237
getting recommendations for user:  238
getting recommendations f

getting recommendations for user:  425
getting recommendations for user:  426
getting recommendations for user:  427
getting recommendations for user:  428
getting recommendations for user:  429
getting recommendations for user:  430
getting recommendations for user:  431
getting recommendations for user:  432
getting recommendations for user:  433
getting recommendations for user:  434
getting recommendations for user:  435
getting recommendations for user:  436
getting recommendations for user:  437
getting recommendations for user:  438
getting recommendations for user:  439
getting recommendations for user:  440
getting recommendations for user:  441
getting recommendations for user:  442
getting recommendations for user:  443
getting recommendations for user:  444
getting recommendations for user:  445
getting recommendations for user:  446
getting recommendations for user:  447
getting recommendations for user:  448
getting recommendations for user:  449
getting recommendations f

getting recommendations for user:  636
getting recommendations for user:  637
getting recommendations for user:  638
getting recommendations for user:  639
getting recommendations for user:  640
getting recommendations for user:  641
getting recommendations for user:  642
getting recommendations for user:  643
getting recommendations for user:  644
getting recommendations for user:  645
getting recommendations for user:  646
getting recommendations for user:  647
getting recommendations for user:  648
getting recommendations for user:  649
getting recommendations for user:  650
getting recommendations for user:  651
getting recommendations for user:  652
getting recommendations for user:  653
getting recommendations for user:  654
getting recommendations for user:  655
getting recommendations for user:  656
getting recommendations for user:  657
getting recommendations for user:  658
getting recommendations for user:  659
getting recommendations for user:  660
getting recommendations f

getting recommendations for user:  847
getting recommendations for user:  848
getting recommendations for user:  849
getting recommendations for user:  850
getting recommendations for user:  851
getting recommendations for user:  852
getting recommendations for user:  853
getting recommendations for user:  854
getting recommendations for user:  855
getting recommendations for user:  856
getting recommendations for user:  857
getting recommendations for user:  858
getting recommendations for user:  859
getting recommendations for user:  860
getting recommendations for user:  861
getting recommendations for user:  862
getting recommendations for user:  863
getting recommendations for user:  864
getting recommendations for user:  865
getting recommendations for user:  866
getting recommendations for user:  867
getting recommendations for user:  868
getting recommendations for user:  869
getting recommendations for user:  870
getting recommendations for user:  871
getting recommendations f

...

getting recommendations for user:  5156
getting recommendations for user:  5157
getting recommendations for user:  5158
getting recommendations for user:  5159
getting recommendations for user:  5160
getting recommendations for user:  5161
getting recommendations for user:  5162
getting recommendations for user:  5163
getting recommendations for user:  5164
getting recommendations for user:  5165
getting recommendations for user:  5166
getting recommendations for user:  5167
getting recommendations for user:  5168
getting recommendations for user:  5169
getting recommendations for user:  5170
getting recommendations for user:  5171
getting recommendations for user:  5172
getting recommendations for user:  5173
getting recommendations for user:  5174
getting recommendations for user:  5175
getting recommendations for user:  5176
getting recommendations for user:  5177
getting recommendations for user:  5178
getting recommendations for user:  5179
getting recommendations for user:  5180


getting recommendations for user:  5361
getting recommendations for user:  5362
getting recommendations for user:  5363
getting recommendations for user:  5364
getting recommendations for user:  5365
getting recommendations for user:  5366
getting recommendations for user:  5367
getting recommendations for user:  5368
getting recommendations for user:  5369
getting recommendations for user:  5370
getting recommendations for user:  5371
getting recommendations for user:  5372
getting recommendations for user:  5373
getting recommendations for user:  5374
getting recommendations for user:  5375
getting recommendations for user:  5376
getting recommendations for user:  5377
getting recommendations for user:  5378
getting recommendations for user:  5379
getting recommendations for user:  5380
getting recommendations for user:  5381
getting recommendations for user:  5382
getting recommendations for user:  5383
getting recommendations for user:  5384
getting recommendations for user:  5385


getting recommendations for user:  5566
getting recommendations for user:  5567
getting recommendations for user:  5568
getting recommendations for user:  5569
getting recommendations for user:  5570
getting recommendations for user:  5571
getting recommendations for user:  5572
getting recommendations for user:  5573
getting recommendations for user:  5574
getting recommendations for user:  5575
getting recommendations for user:  5576
getting recommendations for user:  5577
getting recommendations for user:  5578
getting recommendations for user:  5579
getting recommendations for user:  5580
getting recommendations for user:  5581
getting recommendations for user:  5582
getting recommendations for user:  5583
getting recommendations for user:  5584
getting recommendations for user:  5585
getting recommendations for user:  5586
getting recommendations for user:  5587
getting recommendations for user:  5588
getting recommendations for user:  5589
getting recommendations for user:  5590


getting recommendations for user:  5771
getting recommendations for user:  5772
getting recommendations for user:  5773
getting recommendations for user:  5774
getting recommendations for user:  5775
getting recommendations for user:  5776
getting recommendations for user:  5777
getting recommendations for user:  5778
getting recommendations for user:  5779
getting recommendations for user:  5780
getting recommendations for user:  5781
getting recommendations for user:  5782
getting recommendations for user:  5783
getting recommendations for user:  5784
getting recommendations for user:  5785
getting recommendations for user:  5786
getting recommendations for user:  5787
getting recommendations for user:  5788
getting recommendations for user:  5789
getting recommendations for user:  5790
getting recommendations for user:  5791
getting recommendations for user:  5792
getting recommendations for user:  5793
getting recommendations for user:  5794
getting recommendations for user:  5795


getting recommendations for user:  5976
getting recommendations for user:  5977
getting recommendations for user:  5978
getting recommendations for user:  5979
getting recommendations for user:  5980
getting recommendations for user:  5981
getting recommendations for user:  5982
getting recommendations for user:  5983
getting recommendations for user:  5984
getting recommendations for user:  5985
getting recommendations for user:  5986
getting recommendations for user:  5987
getting recommendations for user:  5988
getting recommendations for user:  5989
getting recommendations for user:  5990
getting recommendations for user:  5991
getting recommendations for user:  5992
getting recommendations for user:  5993
getting recommendations for user:  5994
getting recommendations for user:  5995
getting recommendations for user:  5996
getting recommendations for user:  5997
getting recommendations for user:  5998
getting recommendations for user:  5999
getting recommendations for user:  6000


In [51]:
df

,user_id,recommendations_by_similarity,recommendations_by_genre,recommendations_by_last_viewed_film,recommendations_by_age_gender,recommendations_by_profession
0,1,Raiders of the Lost Ark (1981),Schlafes Bruder (Brother of Sleep) (1995),Song of Freedom (1936),American Beauty (1999),Star Wars: Episode V - The Empire Strikes Back...
1,2,"Godfather, The (1972)",Schlafes Bruder (Brother of Sleep) (1995),Smashing Time (1967),Star Wars: Episode IV - A New Hope (1977),Back to the Future (1985)
2,3,Casablanca (1942),Smashing Time (1967),Follow the Bitch (1998),Saving Private Ryan (1998),Terminator 2: Judgment Day (1991)
3,4,One Flew Over the Cuckoo's Nest (1975),Sanjuro (1962),Seven Samurai (The Magnificent Seven) (Shichin...,American Beauty (1999),Terminator 2: Judgment Day (1991)
4,5,One Flew Over the Cuckoo's Nest (1975),Schlafes Bruder (Brother of Sleep) (1995),One Little Indian (1973),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back...
5,6,Schindler's List (1993),Skipped Parts (2000),Smashing Time (1967),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back...
6,7,"Godfather, The (1972)",Sanjuro (1962),Schlafes Bruder (Brother of Sleep) (1995),American Beauty (1999),Star Wars: Episode IV - A New Hope (1977)
7,8,"Shawshank Redemption, The (1994)",Schlafes Bruder (Brother of Sleep) (1995),Song of Freedom (1936),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back...
8,9,Pulp Fiction (1994),Schlafes Bruder (Brother of Sleep) (1995),Song of Freedom (1936),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back...
9,10,"Killing Fields, The (1984)",Smashing Time (1967),Follow the Bitch (1998),Saving Private Ryan (1998),"Silence of the Lambs, The (1991)"


In [53]:
#save DF to disk
df.to_excel('data/recommendations.xlsx',index = 0)

## All the recommendatsions are now stored in the excel sheet 

### Making the recommendation system interactive 

what good is a recommendation if it’s not interactive and does not learn more about user as user interact with it? 
Thus, I tried to make this recommendations system interactive. 


We will be asked to enter the user id of user we want to get recommendations for 
Here are the things we can do <br>
-	We can get recommendations <br>
-	We watch a film and rate it if we want to<br>
    - If rate it, then all the corresponding records will be updated<br>
    -	If we chose not to rate the film watched, it will be just added to list of films already recommended<br>


In [19]:
def recommendations_per_user(i, current_recommended_movie_id, past_recommendations):
    recommendations_by_similarity = recommend_similarity(i, current_recommended_movie_id, past_recommendations)
    recommendations_by_genre = recommend_genre(i, current_recommended_movie_id, past_recommendations)
    recommendations_by_last_viewed_film = last_watched(i, current_recommended_movie_id, past_recommendations)
    recommendations_by_age_gender = recommend_age_gender(i, current_recommended_movie_id, past_recommendations)
    recommendations_by_profession = recommend_job(i, current_recommended_movie_id, past_recommendations)
        
    r1 = recommendations_by_similarity
    r2 = recommendations_by_genre
    r3 = recommendations_by_last_viewed_film
    r4 = recommendations_by_age_gender0
    r5 = recommendations_by_profession
    t1, t2, t3, t4, t5 = get_title(r1, r2, r3, r4, r5)
    print('recommendations_by_similarity: movie_id:', r1, ', movie title:',t1)
    print('recommendations_by_genre: movie_id:', r2, ', movie title:',t2)
    print('recommendations_by_last_viewed_film: movie_id:', r3, ', movie title:',t3)
    print('recommendations_by_age_gender: movie_id:', r4, ', movie title:',t4)
    print('recommendations_by_profession: movie_id:', r5, ', movie title:',t5)
    return r1,r2,r3,r4,r5

In [57]:
def update_records(current_id,movie_id,rating):
    # delete the cosine distances asssoicated with this user ID
    # update the average ratings
    # update the genre 
    # update the vewing history and job based data
    user_id_ratings_data[current_id][rating].append(movie_id)
    user_id_movie_data[current_id][movie_id] = rating
    movie_based_dict[movie_id][current_id] = rating
    user_id_viewing_history[current_id]['movie_id'] = movie_id
     # updating user viewing history
    genre_of_film = movie_id_data[current_id]['genre']
    movie_id_data[movie_id]['ratings'].append(rating)
    for j in genre_of_film:
        if j not in user_id_viewing_data[current_id].keys():
            user_id_viewing_data[current_id][j] = 1
        else:
            user_id_viewing_data[current_id][j] += 1
    ############################################### 
    age_gender_movies[user_id_info[current_id]['age']][user_id_info[current_id]['gender']][row[1]] += 1
    if user_id_info[current_id]['job'] not in job_data.keys():
        job_data[user_id_info[current_id]['job']] = {}
    if movie_id not in job_data[user_id_info[current_id]['job']].keys():
        job_data[user_id_info[current_id]['job']][movie_id] = {}
        job_data[user_id_info[current_id]['job']][movie_id] = 1
    else:
        job_data[user_id_info[current_id]['job']][movie_id] += 1
    
    movie_id_data[movie_id]['mean'] = np.mean(movie_id_data[movie_id]['ratings'])
    ############################## updating the cosine_similarity_hash
    users_assosiated_current_id = cosine_similarity_hash[current_id].keys()
    for j in users_assosiated_current_id:
        try:
            del cosine_similarity_hash[j][current_id]
        except KeyError:
            pass
    del  cosine_similarity_hash[current_id]    
    print('all clear: records updated')
    return None

In [34]:
def watch_the_film(movie_id,current_id,current_recommended_movie_id):
    print('hope you watched the film!!')
    print('enter 1 to enter rating of the film, to update records')
    print('enter 0 to get main screen')
    enter_val = int(input())
    if enter_val == 1:
        print('enter rating: whole numbers only, range 1 to 5')
        enter_val  = int(input())
        update_records(current_id,movie_id,enter_val)
        print('thanks for the the rating the film\n')
    else:
        current_recommended_movie_id = current_recommended_movie_id.union(set([movie_id]))
    
    

In [35]:
def get_feed_back(current_id, current_recommended_movie_id, past_recommendations,sugessions):
    # watch one of the film?
    # dont recommend these films again? ask for  movie_ids, and removes these movie ids from current recommendations
    print('\npick one of the options below\n######')
    print('enter 1: get new recommendations')
    print('enter 2: watch a film')
    print('enter 0 as  input to got to options on main screen')
    sugessions = set(sugessions)
    enter_val = int(input())
    if enter_val == 1:  
        r1,r2,r3,r4,r5=recommendations_per_user( current_id, current_recommended_movie_id, past_recommendations)
        sugessions = [r1,r2,r3,r4,r5]
        sugessions = list(filter(lambda x : x!='No recommendations in this category',sugessions ))
        get_feed_back( current_id, current_recommended_movie_id, past_recommendations,sugessions) 
    elif enter_val == 2:
        print('enter the movie id of the film  you want to see from the current iteration of recommendations' )
        enter_val  = int(input())
        if enter_val in sugessions:
            sugessions = sugessions.difference([enter_val])
            current_recommended_movie_id = current_recommended_movie_id.difference(sugessions)
            watch_the_film(enter_val,current_id,current_recommended_movie_id)
        else:
            print('entered wrong movie Id so exiting')
    else:
        pass
        
        
        

In [36]:
if __name__ == '__main__':
    print('enter user id to get options')
    current_id = int(input())
    current_recommended_movie_id =set()
    past_recommendations=set()
    while(True):
        print('\npick one of the options\n##########\n')
        print('enter 0 to exit')
        print('enter 1 to get recommendations')

        entered_val = int(input())
        if entered_val == 1:        
            # get recommendations
            r1,r2,r3,r4,r5=recommendations_per_user( current_id, current_recommended_movie_id, past_recommendations)
            sugessions = [r1,r2,r3,r4,r5]
            sugessions = list(filter(lambda x : x!='No recommendations in this category',sugessions ))
            get_feed_back( current_id, current_recommended_movie_id, past_recommendations,sugessions) 
        elif entered_val == 0:
            break
        else:
            print('entered wrong value')
    

enter user id to get options
1
pick one of the options
##########

enter 0 to exit
enter 1 to get recommendations
1
recommendations_by_similarity: movie_id: 1584 , movie title: Contact (1997)
recommendations_by_genre: movie_id: 2 , movie title: Jumanji (1995)
recommendations_by_last_viewed_film: movie_id: 110 , movie title: Braveheart (1995)
recommendations_by_age_gender: movie_id: No recommendations in this category , movie title: No recommendations in this category
recommendations_by_profession: movie_id: No recommendations in this category , movie title: No recommendations in this category
pick one of the options below
######
enter 1: get new recommendations
enter 2: watch a film
enter 0 as  input to got to options on main screen
1
recommendations_by_similarity: movie_id: 2571 , movie title: Matrix, The (1999)
recommendations_by_genre: movie_id: 1282 , movie title: Fantasia (1940)
recommendations_by_last_viewed_film: movie_id: 2291 , movie title: Edward Scissorhands (1990)
recommend

### Comments on the system 

   - The set "past_recommendations" is used to store the values of recommendations that were given to user in the past, that means at different time not in the current interactive session. I have not implemented this feature yet for now. In future I may do it. 
   - When using the interactive aspect of the system, when getting recommendations for the first time, It will take around 10 seconds to get recommendations but if you are interacting for the second time you will get recommendations instantaneously.
   - When one gives rating to the film, we need to recalculate cosine similarity for that user.

### Final notes

Things I wish I could have done to improve the system. 

- Movie year is given as part of movie title, I could have used to recommend only the latest films. 
- Few people user preferences change with time and few does not.  Apparently, I don’t know how to model this kind of aspect. Need to think about it.
- Could have taken use of the multicore processor or GPU to improve the performance. As of now don’t have in depth knowledge on these topics, So I chose not to do it. 
- Was tempted to implement this https://code.facebook.com/posts/861999383875667/recommending-items-to-more-than-a-billion-people/  but since we are asked to implement only a ‘basic recommendation system ‘  I backed off and anyway its way to complex and few algorithms tends to do better only if we are having large data set. 
- Although at first this data set looks large, it is very tiny when compared to data sets people work in real life. I built this system to be scalable, I hope it will be if implemented using help of GPU’s and few fixes here and there, though can’t guarantee as I have not tested it with huge data sets.
- I planned to implement a way to save preprocessed data on the disk, I could not do it. I may do it in future, this will help us save lot of time when restarting the system and saving all the information learned about the user. 
- I do realize there are few areas this system can still be improved. Any sort of criticism of this system is welcomed. 

